<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing Images with Phoenix</h1>

In this tutorial, you will:
- Trace a toy application that uses images, and view those images in Phoenix.

ℹ️ This notebook requires an OpenAI API key.

# Install dependencies

In [1]:
%%bash
pip install -q "arize-phoenix>=4.29.0" openinference-instrumentation-openai openai

# Connect to Phoenix

In [2]:
# Check if PHOENIX_API_KEY is present in the environment variables.
# If it is, we'll use the cloud instance of Phoenix. If it's not, we'll start a local instance.
# A third option is to connect to a docker or locally hosted instance.
# See https://docs.arize.com/phoenix/setup/environments for more information.

import os

if "PHOENIX_API_KEY" in os.environ:
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

else:
    import phoenix as px

    px.launch_app().view()

INFO:phoenix.config:📋 Ensuring phoenix working directory: /Users/jgilhuly/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from phoenix.otel import register

tracer_provider = register()

INFO  [phoenix.server.app] Loading trace fixtures.
INFO  [phoenix.server.app] Finished loading fixtures.


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer=False`.



# Instrument OpenAI

Phoenix's existing auto-instrumentors allow you to capture images by image url with OpenAI

In [4]:
from openinference.instrumentation.openai import OpenAIInstrumentor
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

# Include an image url in an OpenAI prompt

In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This image depicts a scenic landscape featuring a wooden boardwalk path that winds through a grassy field. The tall green grass flanks both sides of the boardwalk, which leads toward a distant horizon where trees and bushes can be seen. The sky above is expansive with scattered clouds, giving the impression of a serene, pleasant day.', refusal=None, role='assistant', function_call=None, tool_calls=None))


In [7]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What do these images have in common?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/7/73/Chicago_Bulls_and_New_Jersey_Nets%2C_March_28%2C_1991.jpg/640px-Chicago_Bulls_and_New_Jersey_Nets%2C_March_28%2C_1991.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/NBA_shot_clock.jpg/640px-NBA_shot_clock.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Kristaps_Porzingis_and_Kris_Humphries.jpg/640px-Kristaps_Porzingis_and_Kris_Humphries.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='All three images depict professional basketball games being played, featuring players in action on the court. The first two images involve basketball teams with players in distinctive jerseys, indicating competitive play during a game. The third image also shows a basketball game with players competing under the basket. Additionally, the New York Knicks are prominently featured in all three images, suggesting they are the common team in each of the games.', refusal=None, role='assistant', function_call=None, tool_calls=None))


You should now see the images in your Phoenix app!